<a href="https://colab.research.google.com/github/paulxiong/simclr/blob/master/SimCLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains a PyTorch implementation of the paper [A Simple Framework for Contrastive Learning of Visual Representations](https://arxiv.org/abs/2002.05709) by Chen et al.

**Try to modify the CLR to train a 100x100 pic data-set.**

In [1]:
import io
import requests
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
GPATH="/content/gdrive/My Drive/maliria/"

MODEL_SIZE = 100

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pathlib
from shutil import copyfile
import os
#MALARIA_PATH="malaria/"
#pathlib.Path(PYIMAGESEARCH_PATH).mkdir(parents=True, exist_ok=True)
#pathlib.Path(MALARIA_PATH).mkdir(parents=True, exist_ok=True)
if not os.path.isfile("cifar-10-python.tar.gz"):
    print("Copying cifar-10-python.tar.gz to local folder")
    copyfile(GPATH+"cifar-10-python.tar.gz", "cifar-10-python.tar.gz")
    print("Extracting cifar-10-python.tar.gz")
    !ls cifar-10-batches-py/
    !rm -rf cifar-10-batches-py
    !ls cifar-10-batches-py/
    os.system("tar -zxvf  "+"cifar-10-python.tar.gz")

Copying cifar-10-python.tar.gz to local folder
Extracting cifar-10-python.tar.gz
ls: cannot access 'cifar-10-batches-py/': No such file or directory
ls: cannot access 'cifar-10-batches-py/': No such file or directory


In [3]:
import os #boostx
import os.path  #boostx
import sys #boostx
#from vision import VisionDataset
#from utils import check_integrity, download_and_extract_archive
from torchvision.datasets.utils import check_integrity, download_and_extract_archive
import pickle #boostx

import numpy as np
from tqdm import tqdm_notebook as tqdm
from PIL import Image

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as tfs
from torchvision.datasets import *
from torchvision.models import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
tf_tr = tfs.Compose([
    tfs.RandomResizedCrop(MODEL_SIZE),
    tfs.RandomHorizontalFlip(),
    tfs.ColorJitter(0.5, 0.5, 0.5, 0.5),
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], 
                  std=[0.229, 0.224, 0.225])
])

tf_de = tfs.Compose([
    tfs.Resize(MODEL_SIZE),
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], 
                  std=[0.229, 0.224, 0.225])
])

tf_te = tfs.Compose([
    tfs.Resize(MODEL_SIZE),
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], 
                  std=[0.229, 0.224, 0.225])
])

In [0]:
class CustomCIFAR10(CIFAR10):
    def __init__(self, **kwds):
        super().__init__(**kwds)
            
    def __getitem__(self, idx):
        if not self.train:
            return super().__getitem__(idx)
    
        img = self.data[idx]
        img = Image.fromarray(img).convert('RGB')
        imgs = [self.transform(img), self.transform(img)]
        return torch.stack(imgs)

In [0]:
class CustomCervicalCIFAR10(CIFAR10):
    base_folder = '/content/cifar-10-batches-py'
    #url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
   
    tgz_md5 = 'c58f30108f718f92721af3b95e74349a'
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]

    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]
    meta = {
        'filename': 'batches.meta',
        'key': 'label_names',
        'md5': '5ff9c542aee3614f3951f8cda6e48888',
    }

    def __init__(self, root, train=True, transform=None, target_transform=None,
                 download=False):
        super(CIFAR10, self).__init__(root, transform=transform,
                                      target_transform=target_transform)
        
        self.train = train  # training set or test set

        if self.train:
            downloaded_list = self.train_list
        else:
            downloaded_list = self.test_list

        self.data = []
        self.targets = []

        # now load the picked numpy arrays
        for file_name, checksum in downloaded_list:
            file_path = os.path.join(self.root, self.base_folder, file_name)
            with open(file_path, 'rb') as f:
                if sys.version_info[0] == 2:
                    entry = pickle.load(f)
                else:
                    entry = pickle.load(f, encoding='latin1')
                self.data.append(entry['data'])
                if 'labels' in entry:
                    self.targets.extend(entry['labels'])
                else:
                    self.targets.extend(entry['fine_labels'])

        self.data = np.vstack(self.data).reshape(-1,3,MODEL_SIZE,MODEL_SIZE)
        self.data = self.data.transpose((0,2,3,1))  #covert to HWC

        self._load_meta()
    
    def _load_meta(self):
        path = os.path.join(self.root, self.base_folder, self.meta['filename'])
        #if not check_integrity(path, self.meta['md5']):
        #    raise RuntimeError('Dataset metadata file not found or corrupted.' +
        #                       ' You can use download=True to download it')
        with open(path, 'rb') as infile:
            if sys.version_info[0] == 2:
                data = pickle.load(infile)
            else:
                data = pickle.load(infile, encoding='latin1')
            self.classes = data[self.meta['key']]
        self.class_to_idx = {_class: i for i, _class in enumerate(self.classes)}

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.targets[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        if self._check_integrity():
            print('Files already downloaded and verified')
            return
        download_and_extract_archive(self.url, self.root, filename=self.filename, md5=self.tgz_md5)

    def extra_repr(self):
        return "Split: {}".format("Train" if self.train is True else "Test")




In [0]:
class C_CustomCervicalCIFAR10(CustomCervicalCIFAR10):
    def __init__(self, **kwds):
        super().__init__(**kwds)
            
    def __getitem__(self, idx):
        if not self.train:
            return super().__getitem__(idx)
    
        img = self.data[idx]
        img = Image.fromarray(img).convert('RGB')
        imgs = [self.transform(img), self.transform(img)]
        return torch.stack(imgs)

**Loading Data: Training, Validate, Test**

In [0]:
#ds_tr = CustomCIFAR10(root='data', train=True, transform=tf_tr, download=True)
ds_tr = C_CustomCervicalCIFAR10(root='data', train=True, transform=tf_tr, download=True)

#boostx ds_de = CIFAR10(root='data', train=True, transform=tf_de, download=True)
ds_de = CustomCervicalCIFAR10(root='data', train=True, transform=tf_de, download=True)

#ds_te = CIFAR10(root='data', train=False, transform=tf_te, download=True)
ds_te = CustomCervicalCIFAR10(root='data', train=False, transform=tf_te, download=True)

In [9]:
'''boostx : due to run out of GPU cuda memory
dl_tr = DataLoader(ds_tr, batch_size=256, shuffle=True)
dl_de = DataLoader(ds_de, batch_size=256, shuffle=True)
dl_te = DataLoader(ds_te, batch_size=256, shuffle=False)'''

'boostx : due to run out of GPU cuda memory\ndl_tr = DataLoader(ds_tr, batch_size=256, shuffle=True)\ndl_de = DataLoader(ds_de, batch_size=256, shuffle=True)\ndl_te = DataLoader(ds_te, batch_size=256, shuffle=False)'

In [0]:
dl_tr = DataLoader(ds_tr, batch_size=16, shuffle=True)
dl_de = DataLoader(ds_de, batch_size=16, shuffle=True)
dl_te = DataLoader(ds_te, batch_size=16, shuffle=False)

In [0]:
model = resnet50(pretrained=False)
model.conv1 = nn.Conv2d(3, 64, 3, 1, 1, bias=False)
model.maxpool = nn.Identity()

In [12]:
ch = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(ch, ch),
                           nn.ReLU(),
                           nn.Linear(ch, ch))
model.to(device)
model.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentu

**相似函数和损失函数**

In [0]:
def pair_cosine_similarity(x, eps=1e-8):
    n = x.norm(p=2, dim=1, keepdim=True)
    return (x @ x.t()) / (n * n.t()).clamp(min=eps)

def nt_xent(x, t=0.5):
    x = pair_cosine_similarity(x)
    x = torch.exp(x / t)
    idx = torch.arange(x.size()[0])
    # Put positive pairs on the diagonal
    idx[::2] += 1
    idx[1::2] -= 1
    x = x[idx]
    # subtract the similarity of 1 from the numerator
    x = x.diag() / (x.sum(0) - torch.exp(torch.tensor(1 / t)))
    return -torch.log(x.mean())

In [0]:
optimizer = Adam(model.parameters(), lr=0.001)

In [0]:
model.train()
#boostx for i in range(100):
EPOCH = 100
if not os.path.isfile(GPATH+'cifar10-rn50-mlp-b256-t0.5-e'+str(EPOCH)+'.pt'):
  for i in range(EPOCH):
    c, s = 0, 0
    pBar = tqdm(dl_tr)
    for data in pBar:
        d = data.size()
        x = data.view(d[0]*2, d[2], d[3], d[4]).to(device)
        optimizer.zero_grad()
        p = model(x)
        loss = nt_xent(p)
        s = ((s*c)+(float(loss)*len(p)))/(c+len(p))
        c += len(p)
        pBar.set_description('Train: '+str(round(float(s),3)))
        loss.backward()
        optimizer.step()
    if (i+1) % 2 == 0:
        torch.save(model.state_dict(), GPATH+'cifar10-rn50-mlp-b256-t0.5-e'+str(i+1)+'.pt')
else:
  model.load_state_dict(torch.load(GPATH+'cifar10-rn50-mlp-b256-t0.5-e'+str(EPOCH)+'.pt'))

In [0]:
for param in model.parameters():
    param.requires_grad = False

In [0]:
model.fc = nn.Linear(ch, len(ds_de.classes))
model.to(device)

In [0]:
optimizer = Adam(model.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()

In [0]:
'''model.train()
for i in range(5):
    c, s = 0, 0
    pBar = tqdm(dl_de)
    for data in pBar:
        x, y = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        p = model(x)
        loss = criterion(p, y)
        s = ((s*c)+(float(loss)*len(p)))/(c+len(p))
        c += len(p)
        pBar.set_description('Train: '+str(round(float(s),3)))
        loss.backward()
        optimizer.step()'''

In [0]:
optimizer = Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [0]:
'''model.train()
for i in range(5):
    c, s = 0, 0
    pBar = tqdm(dl_de)
    for data in pBar:
        x, y = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        p = model(x)
        loss = criterion(p, y)
        s = ((s*c)+(float(loss)*len(p)))/(c+len(p))
        c += len(p)
        pBar.set_description('Train: '+str(round(float(s),3)))
        loss.backward()
        optimizer.step()'''

In [0]:
model.eval()
c, s = 0, 0
pBar = tqdm(dl_te)
for data in pBar:
    x, y, = data[0].to(device), data[1].to(device)
    p = model(x)
    loss = criterion(p, y)
    s = ((s*c)+(float(loss)*len(p)))/(c+len(p))
    c += len(p)
    pBar.set_description('Test: '+str(round(float(s),3)))

In [0]:
model.eval()
y_pred, y_true = [], []
pBar = tqdm(dl_te)
for data in pBar:
    x, y = data[0].to(device), data[1].to(device)
    p = model(x)
    #print(data[0])
    print(len(data[0]),len(data[1]))
    y_pred.append(p.cpu().detach().numpy())
    y_true.append(y.cpu().detach().numpy())
y_pred = np.concatenate(y_pred)
y_true = np.concatenate(y_true)

In [0]:
y_pred

In [0]:
y_true
#print(len(y_true))

In [0]:
dl_te.dataset.transforms

In [0]:
(y_true == y_pred.argmax(axis=1)).mean()
#y_true
#np.sum(outputs==labels)/float(.size)

In [0]:
y_pred.argmax(axis=1)